In [1]:
def get_company_pitchbook_url(name):
    
    # import packages
    from time import sleep
    from selenium import webdriver
    from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
    from selenium.webdriver.support.ui import WebDriverWait
     
    # open driver and website
    driver = webdriver.Chrome('/Applications/chromedriver')

    driver.get('https://pitchbook.com/profiles/company/54716-23#overview')
    
    # search for the company
    driver.find_element_by_name('q').send_keys(name.strip())
    sleep(1)
    try:
        driver.find_element_by_xpath("//button[@class = 'btn-search search-form__submit']").click()
    except:
        driver.find_element_by_xpath("//button[@class = 'btn-search btn-search_collapsed hide-block-XL offset-left-XL-10 offset-right-XL-10 offset-top-XL-5 offset-bottom-XL-5 d-block-S']").click()
    sleep(0.5)
    
    # check if company name is correct, and get url
    url = ''
    for i in range(10):
        try:
            company = driver.find_elements_by_xpath("//a[@class = 'offset-top-XL-15 offset-bottom-XL-15 flex-centered-container']")[i].text
            company = company.split('\n')[0].strip()
            if company.lower() == name.strip().lower():
                url = driver.find_elements_by_xpath("//a[@class = 'offset-top-XL-15 offset-bottom-XL-15 flex-centered-container']")[i].get_attribute('href')
                break
        except:
            pass
    
    # close driver
    driver.close()
    
    return url

In [2]:
def get_company_pitchbook_info(name):
    
    # import
    from urllib.request import urlopen
    from bs4 import BeautifulSoup
    import pandas as pd
    
    # get url
    url = get_company_pitchbook_url(name)
    
    # create dataframe
    df = pd.DataFrame(columns=['Name','Description','Founded','Status','Employees','Latest Deal Type','Latest Deal Amount','Investors','Share Price','Stock Symbol','Investments','Financing Rounds','Website','Ownership Status','Financing Status','Primary Industry','Other Industries','Stock Exchange'])
    df.loc[0] = ''
    df['Name'][0] = name
    
    if url != '': 
        # get webpage source
        content=urlopen(url)
        soup=BeautifulSoup(content,'html.parser')
            
        # fill in information
        description = ''
        try:
            description = soup.find_all('p', class_ = 'primary-dark-text-color primary-font shift-bottom-XL-0')[0].get_text().strip()
        except:
            pass
        df['Description'][0] = description
        for i,item in enumerate(soup.find_all('div', class_ = 'general-info__statistic XL-12 flex-container flex-wrap flex-align-start')[0].select('li')):
            if i%2==0:
                itemname = item.get_text()
            else:
                itemcontent = item.get_text()
                try:
                    df[itemname][0] = itemcontent
                except:
                    pass
        for i in range(len(soup.find_all('div',class_='primary-font primary-dark-text-color semi-font-weight'))):
            iname = soup.find_all('div',class_='shift-bottom-XL-15 shift-bottom-S-10 pp-contact-info_item')[i].select('.semi-font-weight')[0].get_text()
            icontent = ''
            for div in soup.find_all('div',class_='shift-bottom-XL-15 shift-bottom-S-10 pp-contact-info_item')[i].find_all('div',class_='ellipsis-XL'):
                icontent = icontent + '; ' + div.get_text()
            icontent = icontent[2:1000]
            try:
                df[iname][0] = icontent
            except:
                pass

    return df

In [3]:
import pandas as pd

df1 = pd.read_csv('/Users/arlenehuang/Desktop/top pr companies.csv')

In [4]:
# around 2s per company

df2 = pd.DataFrame(columns=['Name','Description','Founded','Status','Employees','Latest Deal Type','Latest Deal Amount','Investors','Share Price','Stock Symbol','Investments','Financing Rounds','Website','Ownership Status','Financing Status','Primary Industry','Other Industries','Stock Exchange'])
for name in df1['Company Name']:
    df = get_company_pitchbook_info(name)
    df2 = pd.concat([df2,df])
df2


,Name,Description,Founded,Status,Employees,Latest Deal Type,Latest Deal Amount,Investors,Share Price,Stock Symbol,Investments,Financing Rounds,Website,Ownership Status,Financing Status,Primary Industry,Other Industries,Stock Exchange
0,360 PR,,,,,,,,,,,,,,,,,
0,Firecracker PR,,,,,,,,,,,,,,,,,
0,Strategic PR Group,,,,,,,,,,,,,,,,,
0,PR Girl Inc,,,,,,,,,,,,,,,,,
0,451 Marketing,,,,,,,,,,,,,,,,,
0,Weiss PR,,,,,,,,,,,,,,,,,
0,Hotwire PR,,,,,,,,,,,,,,,,,
0,Idea Grove,Idea Grove is a technology public relations an...,,,,,,,,,1,,www.ideagrove.com,,,,,
0,Spark,,,,,,,,,,,,,,,,,
0,Clarity,,,,,,,,,,,,,,,,,


In [367]:
df2.to_csv('/Users/arlenehuang/Desktop/Book1.csv')